# Geochronology Calculations

In [18]:
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from bokeh.models import Range1d, LinearAxis, ColumnDataSource, LabelSet, Span, Slope, Label, Legend

from scipy.interpolate import CubicSpline
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
output_notebook()

import geochron_apps as gc

Loading BokehJS ...

<center><img src="images/geochronology.png" align="center">
https://www.explainxkcd.com/wiki/index.php/1829:_Geochronology
</center>

The following presentation shows some of the geochronology calculations learned in the Advanced Geochronology class at University of Saskatchewan taught by Bruce Eglington and Camille Partin, 2021. Some of the images in this presentation are taken from lectures given by the instructor.

This notebook contains sample calculations typically used in geochronology. It can be obtained at https://git.cs.usask.ca/msv275/advanced-geochronology.  
It can be cloned through the git command:  
* git clone https://git.cs.usask.ca/msv275/advanced-geochronology.git


## Sm-Nd Calculations

Start with an appropriate value for depleted mantle at 4570 Ma and calculate and graph the curve for depleted mantle.

Our variables:  
* Decay Constant = 6.54 x 10<sup>-12</sup> (Lugmair and Marti 1978)  
* <sup>143</sup>Nd/<sup>144</sup>Nd<sub>(depleted mantle)</sub> = 0.51320 
* <sup>147</sup>Sm/<sup>144</sup>Nd<sub>(depleted mantle)</sub> = 0.2148  (Reiners et al 2018)
* <sup>143</sup>Nd/<sup>144</sup>Nd<sub>(CHUR)</sub> = 0.512630 
* <sup>147</sup>Sm/<sup>144</sup>Nd<sub>(CHUR)</sub> = 0.1960  (Bouvier et al 2008)

Isochron Equation:

<sup>143</sup>Nd/<sup>144</sup>Nd<sub>(present day)</sub> = <sup>143</sup>Nd/<sup>144</sup>Nd<sub>(initial)</sub> + <sup>147</sup>Sm/<sup>144</sup>Nd<sub>(present day)</sub> * (*e*<sup>λ x t</sup> - 1)

In [19]:
decay_const_143 = 6.54*10**-12
Sm_DM = 0.2148
Nd_DM = 0.51320
Sm_CHUR = 0.1960
Nd_CHUR = 0.512630

d = []
t1, t2 = 4570, 0
while t1 > 0:
    d.append({'t1': t1,
            't2': t2,
            '147Sm/144Nd': Sm_DM, 
            '143Nd/144Nd_DM': gc.calc_initial(Nd_DM, Sm_DM, decay_const_143, t1, t2),
            '143Nd/144Nd_CHUR': gc.calc_initial(Nd_CHUR, Sm_CHUR, decay_const_143, t1, t2),
                 })
    t1 = t1 - 1
SmNd_df = pd.DataFrame(d)

In [20]:
figure10 = gc.get_figure("143Nd/144Nd Depleted Mantle", "Age (Ma)", "143Nd/144Nd", [0,4570], [0.506,0.514])
figure10.line(SmNd_df['t1'], SmNd_df['143Nd/144Nd_DM'], color="darkred", legend_label="Depleted Mantle")

figure10.legend.location = "top_right"
figure10.legend.click_policy="hide"

In [21]:
show(figure10)

Assume a crust generation event at 3000 Ma and another at 500 Ma, each starting from the depleted mantle curve. Assume these produce felsic crust with 147Sm/144Nd values of 0.11. Calculate and graph these two curves plus the curve for CHUR and for depleted mantle. 

In [22]:
c_event1 = 3000
c_event2 = 500
SmNd_event = 0.11
Nd_DM_3000 = SmNd_df[SmNd_df['t1'] == 3000].values[0][-2]
Nd_DM_500 = SmNd_df[SmNd_df['t1'] == 500].values[0][-2]
Nd_CHUR_3000 = SmNd_df[SmNd_df['t1'] == 3000].values[0][-1]
Nd_CHUR_500 = SmNd_df[SmNd_df['t1'] == 500].values[0][-1]

d = []
t1, t2 = 3000, 0
while t2 < 3000:
    d.append({'t1': t1,
            't2': t2,
            '147Sm/144Nd': Sm_DM, 
            '143Nd/144Nd_DM': gc.calc_t2_daughter(Nd_DM_3000, SmNd_event, decay_const_143, t1, t2),
            '143Nd/144Nd_CHUR': gc.calc_t2_daughter(Nd_CHUR_3000, SmNd_event, decay_const_143, t1, t2),
                 })
    t2 = t2 + 1

SmNd_3000_df = pd.DataFrame(d)

d=[]
t1, t2 = 500, 0
while t2 < 500:
    d.append({'t1': t1,
            't2': t2,
            '147Sm/144Nd': Sm_DM, 
            '143Nd/144Nd_DM': gc.calc_t2_daughter(Nd_DM_500, SmNd_event, decay_const_143, t1, t2),
            '143Nd/144Nd_CHUR': gc.calc_t2_daughter(Nd_CHUR_500, SmNd_event, decay_const_143, t1, t2),
                 })
    t2 = t2 + 1   

SmNd_500_df = pd.DataFrame(d)
SmNd_500_df.head()

,t1,t2,147Sm/144Nd,143Nd/144Nd_DM,143Nd/144Nd_CHUR
0,500,0,0.2148,0.512857,0.512348
1,500,1,0.2148,0.512856,0.512348
2,500,2,0.2148,0.512855,0.512347
3,500,3,0.2148,0.512855,0.512346
4,500,4,0.2148,0.512854,0.512345


In [23]:
figure10.line(SmNd_df['t1'], SmNd_df['143Nd/144Nd_CHUR'], color="darkblue", legend_label="CHUR")
#figure10.line(SmNd_3000_df['t2'], SmNd_3000_df['143Nd/144Nd_CHUR'], color="lightblue", legend_label="3000 Ma Event (CHUR)")
figure10.line(SmNd_3000_df['t2'], SmNd_3000_df['143Nd/144Nd_DM'], color="pink", legend_label="3000 Ma Event (DM)")
#figure10.line(SmNd_500_df['t2'], SmNd_500_df['143Nd/144Nd_CHUR'], color="blue", legend_label="500 Ma Event (CHUR)")
figure10.line(SmNd_500_df['t2'], SmNd_500_df['143Nd/144Nd_DM'], color="red", legend_label="500 Ma Event (DM)")

GlyphRenderer(id='2318', ...)

In [24]:
show(figure10)

Assume an igneous rock sample is analysed and provides the following composition:
* 147Sm/144Nd = 0.1280
* 143Nd/144Nd = 0.512000
* The igneous unit has previously been dated at 500 Ma by U-Pb

Calculate the initial composition (ratio and epsilon value). What is its T(DM) and T(2DM)?

In [25]:
Nd143_Nd144_p = 0.512
Sm147_Nd144_p = 0.128
t = 500

initial = gc.calc_initial(Nd143_Nd144_p, Sm147_Nd144_p, decay_const_143, t)
epsilon = gc.calc_epsilon(Nd143_Nd144_p, Nd_CHUR)
t_dm = gc.calc_model_age(Nd143_Nd144_p, Nd_DM, Sm147_Nd144_p, Sm_DM, decay_const_143)
t_dm

2099.417861147067

Assume the crustal evolution history used in the previous assignment and determine possible scenarios for mixing of multiple end-members to explain the composition of the sample at its time of formation. Assume that Nd concentrations in mantle and crust are 0.86 and 22.6, respectively and that Sm concentrations are 0.80 and 22.0, respectively.

In [26]:
figure12 = gc.get_figure("143Nd/144Nd", "Age (Ma)", "143Nd/144Nd", [0,4570], [0.506,0.514])
figure12.line(SmNd_df['t1'], SmNd_df['143Nd/144Nd_DM'], color="darkred", legend_label="Depleted Mantle")
figure12.line(SmNd_df['t1'], SmNd_df['143Nd/144Nd_CHUR'], color="darkblue", legend_label="CHUR")


figure12.legend.location = "top_right"
figure12.legend.click_policy="hide"

In [27]:
show(figure12)

Illustrate the scenarios graphically in Excel for both ratio and epsilon situations.